In [1]:
import csv
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from pyslabgrid import reckoner as reck
from pyslabgrid import mat as mat
from pyslabgrid import eqcat as eqc

In [2]:
def within_interfacezone(evlon, evlat):
    # test the interpolatiors
    hik_finterp = np.load('../01subduction-model/finterp/hik_finterp.npy', allow_pickle=True)[()]
    pbounds = hik_finterp['xbounds']
    if pbounds.contains(Point(evlon, evlat)):
        return 'hik'
    
    puy_finterp = np.load('../01subduction-model/finterp/puy_finterp.npy', allow_pickle=True)[()]
    pbounds = puy_finterp['xbounds']
    if pbounds.contains(Point(evlon, evlat)):
        return 'puy'
    
    return None

# extract events on a profile/box
def get_boxpoints(x1,y1):
    x11,y11 = reck.reckon(x1,y1, 20, 220)
    xm, ym = [],[]
    for dd in range(0,450,40):
        tx,ty = reck.reckon(x11,y11, dd, 220+90)
        xm.append(tx)
        ym.append(ty)
    x2,y2 = reck.reckon(x1,y1, 40, 220)
    x3,y3 = reck.reckon(x2,y2, 500, 220+90)
    x4,y4 = reck.reckon(x3,y3, -40, 220)
    xs = [x1,x2,x3,x4,x1]
    ys = [y1,y2,y3,y4,y1]
    return xs, ys, xm, ym

def get_stats(kmag, kisslab, klon = None, klat=None, szone=None):
    ntMge3 = sum([m>=3.0 for m in xmag])
    ntMge4 = sum([m>=4.0 for m in xmag])
    ntMge5 = sum([m>=5.0 for m in xmag])
    if szone is None:    
        print('Combined subduction  |    %d    |   %d    |    %d    |' %(ntMge3, ntMge4, ntMge5))
        return
    elif szone =='slabs':
        nMge3 = len([x for x,m in zip(kisslab,kmag) if (m>=3.0) & (x==1)])
        nMge4 = len([x for x,m in zip(kisslab,kmag) if (m>=4.0) & (x==1)])
        nMge5 = len([x for x,m in zip(kisslab,kmag) if (m>=5.0) & (x==1)])
        
        crat3, crat4, crat5 = nMge3/ntMge3, nMge4/ntMge4, nMge5/ntMge5
        strtxt = 'Combined intraslab'
    else:
        s_mag, s_isslab = [],[]
        for x,y, m, iss in zip(klon, klat, kmag, kisslab):
            if within_interfacezone(x, y)== szone:
                s_mag.append(m) 
                s_isslab.append(iss)
        nMge3 = len([x for x,m in zip(s_isslab,s_mag) if (m>=3.0) & (x==1)])
        nMge4 = len([x for x,m in zip(s_isslab,s_mag) if (m>=4.0) & (x==1)])
        nMge5 = len([x for x,m in zip(s_isslab,s_mag) if (m>=5.0) & (x==1)])
        
        crat3, crat4, crat5 = nMge3/ntMge3, nMge4/ntMge4, nMge5/ntMge5
        
        if szone=='hik':
            strtxt = 'Hikurangi intraslab'
        else:
            strtxt = 'Puysegur intraslab'
            
    print('%s   |  %d (%.2f) | %d (%.2f) | %d (%.2f)  |' \
                  %(strtxt, nMge3, crat3, nMge4, crat4, nMge5,crat5))


In [3]:
catalogues = ['NZeqcat_Rollins13042020-subd-slab.csv',
             'Grue_NZeqcat_Rollins13042020SepiShallow_subd-slab.csv',
             'Grue_NZeqcat_Rollins13042020SepiDeep-subd.csv',]
folder = 'classified_catalogs/'

In [4]:
ecat = eqc.read_slabcatalogue(folder+catalogues[0])
lon, lat, mag, dep = ecat['lon'], ecat['lat'], ecat['mag'], ecat['dep']
isslab, year = ecat['isslab'], ecat['year']

# let get year >=1900
xlon, xlat, xmag, xisslab, xyear = [], [], [], [],[]

for x,y, m, z, iss, yr in zip(lon, lat, mag, dep, isslab, year):
    if z>300:
        continue
    if yr>=1965:
        xlon.append(x)
        xlat.append(y)
        xmag.append(m)
        xisslab.append(iss)
        xyear.append(yr)

print('                     |  Mw>=3.0 | Mw>=4.0 | Mw>=5.0  |\n')

get_stats(xmag, xisslab, szone=None)
get_stats(xmag, xisslab, szone='slabs')
get_stats(xmag, xisslab, klon=xlon, klat = xlat, szone='hik')
get_stats(xmag, xisslab, klon=xlon, klat = xlat, szone='puy')

                     |  Mw>=3.0 | Mw>=4.0 | Mw>=5.0  |

Combined subduction  |    45955    |   4717    |    455    |
Combined intraslab   |  27838 (0.61) | 2542 (0.54) | 235 (0.52)  |
Hikurangi intraslab   |  25261 (0.55) | 2246 (0.48) | 213 (0.47)  |
Puysegur intraslab   |  2577 (0.06) | 296 (0.06) | 22 (0.05)  |
